# Test call Terra api with service account

The following tasks have already been completed. To replicate this analysis with a different service account or project you may need to perform some / all of these tasks. 

- create service account
 - https://cloud.google.com/iam/docs/creating-managing-service-accounts#creating_a_service_account
- create credentials file
 - https://cloud.google.com/storage/docs/authentication#generating-a-private-key
- run script to register service account with Terra..
 - https://github.com/broadinstitute/terra-tools/tree/master/scripts/register_service_account
 - amend relative path to credentials file to absolute path in docker run command provided above. ie..
 - -v /home/mitchac/credentials.json:/svc.json 
- Grant your service account 'writer' permissions on your workspace.
- Upload the credentials.json file for your service account to files area in this workspace.
- Grant your service account 'reader' permissions on the relevant methods in the Broad Methods Repository.

Copy the credentials.json file to the disk of your notebook instance.. 

In [1]:
!gsutil cp gs://fc-833c2d81-556a-4c83-aed7-21f884f6fec0/notebooks/credentials.json .
!ls

Copying gs://fc-833c2d81-556a-4c83-aed7-21f884f6fec0/notebooks/credentials.json...
/ [1 files][  2.3 KiB/  2.3 KiB]                                                
Operation completed over 1 objects/2.3 KiB.                                      
Config-and-run-workflow.ipynb	  credentials.json
Cost-estimator-gbp-summary.ipynb  runtest.ipynb


Authenticate your service account..

In [1]:
!gcloud auth activate-service-account --key-file credentials.json
!gcloud auth list

Activated service account credentials for: [terra-api@maximal-dynamo-308105.iam.gserviceaccount.com]
                              Credentialed Accounts
ACTIVE  ACCOUNT
        pet-114473713848060875587@firstterrabillingaccount.iam.gserviceaccount.com
*       terra-api@maximal-dynamo-308105.iam.gserviceaccount.com

To set the active account, run:
    $ gcloud config set account `ACCOUNT`



In [2]:
import requests
import os
import json

Check Terra/firecloud api by requesting user details. NB you will authenticate the request with your service account.

In [3]:
token = os.popen('gcloud auth --account=terra-api@maximal-dynamo-308105.iam.gserviceaccount.com print-access-token').read().rstrip()
head = {'accept': 'application/json','Authorization': 'Bearer {}'.format(token)}
myUrl = 'https://api.firecloud.org/me?userDetailsOnly=false'
response = requests.get(myUrl, headers=head)
print(response.json())

{'enabled': {'allUsersGroup': True, 'google': True, 'ldap': True}, 'userInfo': {'userEmail': 'terra-api@maximal-dynamo-308105.iam.gserviceaccount.com', 'userSubjectId': '103764540142056505135'}}


Create an example workflow configuration.. 

In [4]:
token = os.popen('gcloud auth --account=terra-api@maximal-dynamo-308105.iam.gserviceaccount.com print-access-token').read().rstrip()
head = {'accept': '*/*',"Content-Type": "application/json", 'Authorization': 'Bearer {}'.format(token)}

data={
	"name": "internet-speed-test",
	"methodRepoMethod": {
		"methodName": "internet-speed-test",
		"methodVersion": 4,
		"methodNamespace": "mitchac"
	},
	"outputs": {
		"internet_speed_test_wf.downloaded_file_out": "this.dl_out",
		"internet_speed_test_wf.speed_test_results": "this.speedtest_out"
	},
    	"inputs": {
	},
	"rootEntityType": "dl_test",
	"prerequisites": {},
	"methodConfigVersion": 1,
	"deleted": False,
	"namespace": "config-namespace-test"
}

myUrl = 'https://api.firecloud.org/api/workspaces/firstterrabillingaccount/singlem-pilot-2/method_configs/config-namespace-test/internet-speed-test'
response = requests.put(myUrl, data=json.dumps(data), headers=head)
print(response.json())

{'extraInputs': [], 'invalidInputs': {}, 'invalidOutputs': {}, 'methodConfiguration': {'deleted': False, 'inputs': {}, 'methodConfigVersion': 1, 'methodRepoMethod': {'methodName': 'internet-speed-test', 'methodVersion': 4, 'methodNamespace': 'mitchac', 'methodUri': 'agora://mitchac/internet-speed-test/4', 'sourceRepo': 'agora'}, 'name': 'internet-speed-test', 'namespace': 'config-namespace-test', 'outputs': {'internet_speed_test_wf.downloaded_file_out': 'this.dl_out', 'internet_speed_test_wf.speed_test_results': 'this.speedtest_out'}, 'prerequisites': {}, 'rootEntityType': 'dl_test'}, 'missingInputs': [], 'validInputs': ['internet_speed_test_wf.speed_test_task.dockerImage'], 'validOutputs': ['internet_speed_test_wf.downloaded_file_out', 'internet_speed_test_wf.speed_test_results']}


Run a workflow.. 

In [6]:
token = os.popen('gcloud auth --account=terra-api@maximal-dynamo-308105.iam.gserviceaccount.com print-access-token').read().rstrip()
head = {'accept': '*/*',"Content-Type": "application/json", 'Authorization': 'Bearer {}'.format(token)}

data={
  "methodConfigurationNamespace": "singlem",
  "methodConfigurationName": "singlem-single-task",
  "entityType": "autotest_sample",
  "entityName": "SRR3984954",
  "useCallCache": True,
  "deleteIntermediateOutputFiles": True,
  "useReferenceDisks": True,
  "workflowFailureMode": "NoNewCalls"
}

myUrl = 'https://api.firecloud.org/api/workspaces/firstterrabillingaccount/singlem-pilot-2/submissions'
response = requests.post(myUrl, data=json.dumps(data), headers=head)
print(response.json())

{'header': {'entityType': 'autotest_sample', 'inputExpressions': [{'expression': 'this.sra_accession', 'wdlName': 'SingleM_SRA.SRA_accession_num'}, {'expression': 'true', 'wdlName': 'SingleM_SRA.GCloud_Paid'}, {'expression': '"gcp-cp aws-http prefetch"', 'wdlName': 'SingleM_SRA.Download_Method_Order'}, {'expression': '"gs://fc-833c2d81-556a-4c83-aed7-21f884f6fec0/sa-private-key.json"', 'wdlName': 'SingleM_SRA.GCloud_User_Key_File'}, {'expression': 'this.metagenome_size_in_gbp', 'wdlName': 'SingleM_SRA.metagenome_size_in_gbp'}]}, 'request': {'deleteIntermediateOutputFiles': True, 'entityName': 'SRR3984954', 'entityType': 'autotest_sample', 'memoryRetryMultiplier': 1.0, 'methodConfigurationName': 'singlem-single-task', 'methodConfigurationNamespace': 'singlem', 'useCallCache': True, 'useReferenceDisks': True, 'workflowFailureMode': 'NoNewCalls'}, 'status': 'Submitted', 'submissionDate': '2021-06-10T03:27:58.970Z', 'submissionId': '0d1d984a-d51b-45fb-8d77-72d9d81cb0eb', 'submitter': 'terr